# __Eksploracja danych__
## __Projekt 4: Asocjacyjne reguły grupowania__  
## __Karolina Węglowska, Krzysztof Stawarz__  

In [1]:
clc; clear; close all;
data = readmatrix('ed-p04.txt');

head(data)

     2     8    10    20    27    29    31    79    80    99   NaN   NaN
     5    19    30    43    44    64    70    73    79    87   NaN   NaN
     0     6    18    39    55    70    71    79    89    98   NaN   NaN
     1     2    29    32    33    51    56    72    85    95   NaN   NaN
    16    17    24    26    45    55    70    71    79   NaN   NaN   NaN
     6    14    20    25    33    40    51    54    72    88   NaN   NaN
     9    30    32    43    46    78    86    93    99   NaN   NaN   NaN
     4     5    41    55    56    60    78    79    81    88   NaN   NaN



In [2]:
numbers = rmmissing(unique(data));

transactions = zeros(size(data, 1), numel(numbers));
for x=1:size(data, 1)
    
    row = rmmissing(data(x, :));
    
    for y=1:size(row, 2)
        
        val = row(y);
        
        transactions(x, val+1) = transactions(x, val+1) + 1;
    end
    
end
 head(transactions(:, 1:11))

     0     0     1     0     0     0     0     0     1     0     1
     0     0     0     0     0     1     0     0     0     0     0
     1     0     0     0     0     0     1     0     0     0     0
     0     1     1     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     1     0     0     0     0
     0     0     0     0     0     0     0     0     0     1     0
     0     0     0     0     1     1     0     0     0     0     0



In [3]:
%  fajne thresholdy:
% min_supp = 0.0149;
min_supp = 0.005;
% min_supp = 0.003;
% min_supp = 0.002;

iteration = 0;

overall_time = 0;

while 1
    
    iteration = iteration + 1;
    disp(sprintf("\n--------------ITERATION %d--------------", iteration));
    
    %  creating new subsets
    tic
    if iteration==1
        c = rmmissing(unique(data));
    else
        tic
        possible_subsets=[];
        for i=1:size(data,1)

            row = rmmissing(data(i, :));

            sets = nchoosek(row, iteration);

            possible_subsets = [possible_subsets; sets];
        end

        c = unique(sort(possible_subsets, 2), 'rows');
        %c = nchoosek(unique(f), iteration);
    end
    
    time_creating_itemset = toc;
    disp(sprintf("✅ Creating itemset : %.2fs", time_creating_itemset));
    
    if isempty(c)
        disp(sprintf("\nbreak1 - new itemset empty after trying to create new with %d items"), iteration);
        valid_subsets = f;
        break;
    end
    
    % TODO: IMPROVE PRUNING TIME-EFFICIENCY
    % pruning
    if iteration > 1
        tic;
        
        is_in_f = [];
        for i=1:size(c,1)
            row = c(i, :);
            subsets = nchoosek(row, iteration-1);
            is_in_f = [is_in_f; all(ismember(subsets, f, 'rows'))];
        end
        temp = c(is_in_f==1, :);
        
        time_pruning = toc;
        disp(sprintf("✅ Pruning : %.2fs", time_pruning));
        
        if isempty(temp)
            disp(sprintf("\nbreak2 - itemset empty after prunning"));
            valid_subsets = f;
            break;
        end
        c = temp;
    else
        time_pruning = 0.000;
    end
    
    
    % checking the supports and removing subsets below threshold
    tic;
    supports = [];
    for i=1:size(c, 1)
        c_row = c(i, :);
        support = sum(all(transactions(:,c_row+1), 2));
        supports = [supports, support];
    end
    supports = supports / size(data, 1);
    temp1 = c(supports>=min_supp, :);
    
    time_calculating_support = toc;
    disp(sprintf("✅ Calculating support, creating frequentset : %.2fs", time_calculating_support));
    
    if isempty(temp1)
        disp(sprintf("\nbreak3 - frequentset empty after calculating supports"));
        valid_subsets = f;
        break;
    end
    
    f = temp1;
    
    overall_time = overall_time + time_creating_itemset + time_pruning + time_calculating_support;
end

disp(sprintf("\nOveral algorithm time for min_supp = %.3f : %.2fs", min_supp, overall_time));

valid_subsets


--------------ITERATION 1--------------
✅ Creating itemset : 0.01s
✅ Calculating support, creating frequentset : 0.00s

--------------ITERATION 2--------------
✅ Creating itemset : 0.10s
✅ Pruning : 0.55s
✅ Calculating support, creating frequentset : 0.05s

--------------ITERATION 3--------------
✅ Creating itemset : 0.12s
✅ Pruning : 17.56s
✅ Calculating support, creating frequentset : 0.29s

--------------ITERATION 4--------------
✅ Creating itemset : 0.17s
✅ Pruning : 12.19s

break2 - itemset empty after prunning

Overal algorithm time for min_supp = 0.005 : 18.69s

valid_subsets =

     3    64    99
    13    51    62
    15    22    51
    16    39    98
    20    49    75
    22    51    76
    25    90    96
    32    45    51
    33    41    58
    45    71    82
    52    58    71
    59    63    73



In [4]:
odp = valid_subsets;

for i=1:size(odp,1)
    disp("____________________")
    subset = odp(i, :);
    k = size(odp, 2);
    disp(subset);
    while k > 0
        k=k-1;
        disp(nchoosek(subset, k));
    end
end

____________________
     3    64    99

     3    64
     3    99
    64    99

     3
    64
    99

____________________
    13    51    62

    13    51
    13    62
    51    62

    13
    51
    62

____________________
    15    22    51

    15    22
    15    51
    22    51

    15
    22
    51

____________________
    16    39    98

    16    39
    16    98
    39    98

    16
    39
    98

____________________
    20    49    75

    20    49
    20    75
    49    75

    20
    49
    75

____________________
    22    51    76

    22    51
    22    76
    51    76

    22
    51
    76

____________________
    25    90    96

    25    90
    25    96
    90    96

    25
    90
    96

____________________
    32    45    51

    32    45
    32    51
    45    51

    32
    45
    51

____________________
    33    41    58

    33    41
    33    58
    41    58

    33
    41
    58

____________________
    45    71    82

    45    71
    45    82
    71